# Map Area

I chose Honolulu as it has a beautiful Chinese name.

https://mapzen.com/data/metro-extracts/metro/honolulu_hawaii/

In [1]:
import xml.etree.ElementTree as ET
import pprint
import re
import codecs
import json

import os

hawaii = "honolulu_hawaii.osm"


# 1. Data Audit and Problems Encountered

## 1.1 Street Name

The street names have the following problems:

- Some of them used abbreviations.
- Some of them used lower cases.
- Some names include things other than street names, such as "Pualei Cir, Apt 106" and 'Kaelepulu Dr, Kailua,'
- Two records 'South King' and 'King' are likely to miss a "Street" in the end.
- The street types of 'Pali Momi' and 'Ala Ike' are unknown.

The problematic street names are shown as follows.


In [2]:
from collections import defaultdict

street_type_re = re.compile(r'\b\S+\.?$', re.IGNORECASE)
street_types = defaultdict(int)

expected = ["Avenue", "Boulevard", "Commons", "Court","Drive", "Lane", \
            "Parkway","Place","Road", "Square", "Street", \
            "Trail", "Highway", "Way", "Walk", "Terrace", \
           "Center", "Mall", "Circle", "Loop"]

def audit_street_type(street_types, street_name):
    """check if a street name ended with any of the expected types stated above"""
    m = street_type_re.search(street_name)
    if m:
        street_type = m.group()
        if street_type not in expected:
            street_types[street_type].add(street_name)

def is_street_name(elem):
    """check if an element is a street name"""
    return (elem.attrib['k'] == "addr:street")

def audit1(osmfile):
    """conduct the audit on the street names"""
    osm_file = open(osmfile, "r")
    street_types = defaultdict(set)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_street_name(tag):
                    audit_street_type(street_types, tag.attrib['v'])

    return street_types


In [3]:
audit1(hawaii)

defaultdict(set,
            {'106': {'Pualei Cir, Apt 106'},
             'Ave': {'Kalakaua Ave'},
             'Blvd': {'Ala Moana Blvd'},
             'Dr': {'Kipapa Dr'},
             'Hwy': {'Kamehameha Hwy'},
             'Ike': {'Ala Ike'},
             'Kailua,': {'Kaelepulu Dr, Kailua,'},
             'King': {'South King'},
             'Momi': {'Pali Momi'},
             'Pkwy': {'Meheula Pkwy'},
             'St': {'Ala Pumalu St', 'Lusitania St'},
             'St.': {'Lusitania St.'},
             'highway': {'kanehameha highway'},
             'king': {'king'}})

The street names are updated as follows.

In [4]:
MAP = {'Ave'  : 'Avenue',
           'Blvd' : 'Boulevard',
           'Cir'  : "Circle",
           'Dr'   : 'Drive',
           'Ln'   : 'Lane',
           'Pkwy' : 'Parkway',
           'St'   : 'Street',
           'St.'   : 'Street',     
           'Hwy'  : 'Highway',
           'Sq'   : "Square",
           'highway': 'Highway'}

def update_street_name(street_name, map = MAP):
    if street_name == "Pualei Cir, Apt 106":
        street_name = "Apt 106 Pualei Circle"
    
    elif street_name == "Kaelepulu Dr, Kailua,":
        street_name = "Kaelepulu Drive"
    
    elif street_name == "king":
        street_name = "King Street"
    
    elif street_name == "South King":
        street_name = "South King Street"
        
    else:
        street_name = street_name.split(' ')
        type = street_name[-1]
        if type in map:
            street_name[-1] = map[type]
        street_name = ' '.join(street_name)
        street_name = street_name.title()
    
    return street_name

def better_name_check(OSMFILE):
    street_types = audit1(OSMFILE)
    for st_type, ways in street_types.iteritems():
        for name in ways: 
            better_name = update_street_name(name, map = MAP)
            print name, "=>", better_name

In [5]:
better_name_check(hawaii)

king => King Street
South King => South King Street
Pualei Cir, Apt 106 => Apt 106 Pualei Circle
Lusitania St. => Lusitania Street
Meheula Pkwy => Meheula Parkway
Kaelepulu Dr, Kailua, => Kaelepulu Drive
Pali Momi => Pali Momi
Ala Ike => Ala Ike
Kamehameha Hwy => Kamehameha Highway
Ala Moana Blvd => Ala Moana Boulevard
Kalakaua Ave => Kalakaua Avenue
Ala Pumalu St => Ala Pumalu Street
Lusitania St => Lusitania Street
Kipapa Dr => Kipapa Drive
kanehameha highway => Kanehameha Highway


## 1.2 Audit postal codes

The postal codes in Hawaii are expected to be five-digit numbers starting with "967" or "968". 

Auditing the postal codes show that some of the postal codes used zip5 + 4 digits format, which is also acceptable.

The only exepction is "HI 96819", in which "HI " should be deleted.

The postal codes that include more than five-digit number starting with "967" and "968" are as follows.

In [11]:
def audit_postal_code(invalid_postal_codes, postal_code):
    """check if a postal code follows the desired format"""
    try:
        if len(postal_code) != 5 and (postal_code[:3] != '967' or '968'):
            raise ValueError
    except ValueError:
        invalid_postal_codes[postal_code] += 1

def is_postal_code(elem):
    """check if an element is a post code"""
    return elem.attrib['k'] == "addr:postcode"

def audit2(osmfile):
    """conduct the audit on the postal codes"""
    osm_file = open(osmfile, "r")
    invalid_postal_codes = defaultdict(int)
    for event, elem in ET.iterparse(osm_file, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_postal_code(tag):
                    audit_postal_code(invalid_postal_codes, tag.attrib['v'])
    return invalid_postal_codes

In [12]:
audit2(hawaii)

defaultdict(int,
            {'96712-9998': 1,
             '96734-9998': 1,
             '96815-2518': 1,
             '96815-2830': 1,
             '96815-2834': 2,
             '96817-1713': 1,
             '96825-9998': 1,
             '96826-4427': 1,
             'HI 96819': 1})

## 1.3 Adudit phone numbers

Audit shows that the formats of the phone numbers are very inconsistent. 
Some use "-", "." or "()" to seperate digits while others only use space. 
Some have the country code while others don't.
Some records have multiple phone numbers.

In [13]:
def audit_phone_number(invalid_phone_numbers, phone_number):
    """ check if the phone number follows the desired format"""
    try:
        if phone_number[:6] != '+1 808':
            raise ValueError

    except ValueError:
        invalid_phone_numbers[phone_number] += 1

def is_phone_number(elem):
    """ check if an element is a phone number"""
    
    return elem.attrib['k'] == "phone"

def audit3(osmfile):
    """ conduct the audit of the phone numbers"""
    
    osm = open(osmfile, "r")
    invalid_phone_numbers = defaultdict(int)
    for event, elem in ET.iterparse(osm, events=("start",)):
        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_phone_number(tag):
                    audit_phone_number(invalid_phone_numbers, tag.attrib['v'])
    return invalid_phone_numbers

In [14]:
audit3(hawaii)

defaultdict(int,
            {'(808) 486-5100': 1,
             '(808) 492-1637': 1,
             '(808) 536-1330': 2,
             '(808) 625-0411': 1,
             '(808) 668-7367': 1,
             '(808) 734-9000': 1,
             '(808) 769-6921': 1,
             '(808) 845-9498': 1,
             '(808) 924-3303': 1,
             '(808) 926-9717': 1,
             '(808) 955-6329': 1,
             '(808)9242233': 1,
             '+1 (808) 733-0277': 1,
             '+1 (808) 733-1540': 1,
             '+1 (808) 946-0253': 1,
             '+1 323 423 6076': 1,
             '+1 800 463 3339': 1,
             '+1 888-389-3199': 1,
             '+1-808-532-8700': 1,
             '+1-808-545-3008': 1,
             '+1-808-831-4820': 1,
             '+1-808-836-7665': 1,
             '+1-808-836-9828': 1,
             '+1-808-839-6306': 1,
             '+1-808-892-1820': 1,
             '+1-808-922-4911': 1,
             '+1-808-9266162': 1,
             '+1-808-954-7000': 1,
            

Some of the phone numbers are cleaned as follows.

In [8]:
def update_phone_number(phone_number):
    """
    update the phone number to follow the "+1 123 12345678" format
    """
    number = phone_number.replace("+","").replace("-", "").replace(".","").replace("(", "").replace(")","").replace(" ", "")
    if number[0:4] == '1808':
        number = "+1 808 "+number[4:]
    elif number[0:3] == "808":
        number = "+1 808 "+number[3:]
    elif number[0] == "1":
        number = "+1 "+number[1:4]+" "+number[4:]

    return number

def better_phone_check(OSMFILE):
    """display the original phone number and the formatted phone number"""
    numbers = audit3(OSMFILE)
    for number in numbers:
        better_number = update_phone_number(number)
        print number, "=>", better_number

In [9]:
better_phone_check(hawaii)

(808) 924-3303 => +1 808 9243303
8089231234 => +1 808 9231234
+1-808-545-3008 => +1 808 5453008
(808) 769-6921 => +1 808 7696921
+18089234852 => +1 808 9234852
(808) 955-6329 => +1 808 9556329
(808) 625-0411 => +1 808 6250411
808-923-7024 => +1 808 9237024
+1-808-922-4911 => +1 808 9224911
808-831-6831 => +1 808 8316831
+18089221544 => +1 808 9221544
+1 (808) 946-0253 => +1 808 9460253
(808) 926-9717 => +1 808 9269717
+1 800 463 3339 => +1 800 4633339
+1-808-836-7665 => +1 808 8367665
808-922-8838 => +1 808 9228838
+1-808-892-1820 => +1 808 8921820
808-637-7472 => +1 808 6377472
+1-808-532-8700 => +1 808 5328700
808-486-2167 => +1 808 4862167
1-808-955-7470 => +1 808 9557470
+1-808-831-4820 => +1 808 8314820
8088458044 => +1 808 8458044
(808) 734-9000 => +1 808 7349000
808-591-2513 => +1 808 5912513
1 (808) 677-3335 => +1 808 6773335
808-637-7710 => +1 808 6377710
808-625-0320 => +1 808 6250320
18089264167 => +1 808 9264167
+1 (808) 733-0277 => +1 808 7330277
808-637-6241 => +1 808 637

# Converting xml to csv and importing csv to database

In [19]:
#Converting xml to csv
import data
reload(data)
data.process_map(hawaii, validate = True)

In [3]:
#importing csv to database
import csv 
import sqlite3

In [21]:
!wget https://gist.githubusercontent.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f/raw/c03182a6aadda4ae149ba59bf9e39e80833454a9/data_wrangling_schema.sql

--2017-01-31 22:17:31--  https://gist.githubusercontent.com/swwelch/f1144229848b407e0a5d13fcb7fbbd6f/raw/c03182a6aadda4ae149ba59bf9e39e80833454a9/data_wrangling_schema.sql
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 151.101.100.133
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|151.101.100.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 832 [text/plain]
Saving to: ‘data_wrangling_schema.sql.1’

data_wrangling_sche 100%[===================>]     832  --.-KB/s    in 0s      

2017-01-31 22:17:31 (214 MB/s) - ‘data_wrangling_schema.sql.1’ saved [832/832]



In [22]:
with open("data_wrangling_schema.sql") as f:
    create_statements = f.readlines()

In [5]:
con = sqlite3.connect("Cleaned SQL/hawaii.db")
cur = con.cursor()

In [ ]:
con = sqlite3.connect("Cleaned SQL/hawaii.db")
cur = con.cursor()
    
def create_tables():
    """create tables in csv"""
   
    cur.execute("""CREATE TABLE nodes (
    id INTEGER PRIMARY KEY NOT NULL,
    lat FLOAT,
    lon FLOAT,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
)""")

    cur.execute("""CREATE TABLE nodes_tags (
            id INTEGER,
            key TEXT,
            value TEXT,
            type TEXT,
            FOREIGN KEY (id) REFERENCES nodes(id)
        )""")

    cur.execute("""CREATE TABLE ways (
    id INTEGER PRIMARY KEY NOT NULL,
    user TEXT,
    uid INTEGER,
    version TEXT,
    changeset INTEGER,
    timestamp TEXT
);""")

    cur.execute("""CREATE TABLE ways_tags (
    id INTEGER NOT NULL,
    key TEXT NOT NULL,
    value TEXT NOT NULL,
    type TEXT,
    FOREIGN KEY (id) REFERENCES ways(id)
);""")

    cur.execute("""CREATE TABLE ways_nodes (
    id INTEGER NOT NULL,
    node_id INTEGER NOT NULL,
    position INTEGER NOT NULL,
    FOREIGN KEY (id) REFERENCES ways(id),
    FOREIGN KEY (node_id) REFERENCES nodes(id)
);""")

create_tables()

sqlite>.mode csv

sqlite>.import nodes.csv nodes

sqlite>.import nodes_tags.csv nodes_tags

sqlite>.import ways.csv ways

sqlite>.import ways_tags.csv ways_tags

sqlite>.import ways_nodes.csv ways_nodes

# 2. Data Overview with SQL

## Size of the files

honolulu_hawaii.osm 54.2MB

hawaii.db 29.3MB

nodes.csv 21.1MB

nodes_tags.csv 508.2kB

ways.csv 1.6MB

ways_nodes.csv 7.1MB

ways_tags.csv 3.7MB

## Number of nodes

In [7]:
def number_of_nodes():
    """count the number of nodes"""
    result = cur.execute('SELECT COUNT(*) FROM nodes')
    return result.fetchone()[0]

number_of_nodes()

248609

## Number of ways

In [8]:
def number_of_ways():
    """count the number of ways"""
    result = cur.execute('SELECT COUNT(*) FROM ways')
    return result.fetchone()[0]
number_of_ways()

26276

## Number of unique users

In [9]:
def number_of_unique_users():
    """count the number of unique users"""
    result = cur.execute("""SELECT COUNT(DISTINCT(e.uid))
            FROM (SELECT uid FROM nodes UNION ALL SELECT uid FROM ways) e""")
    return result.fetchone()[0]
number_of_unique_users()

471

## Top 10 contributing users

In [21]:
def top_contributing_users():
    """count the top 10 users based on their number of contributions"""
    users = []
    for row in cur.execute("""SELECT e.user, COUNT(*)
            as num FROM (SELECT user
            FROM nodes UNION ALL SELECT user FROM ways) e
            GROUP BY e.user 
            ORDER BY num DESC 
            LIMIT 10"""):
        #print row[0],":", row[1]
        users.append(row)
    return users
top_contributing_users()

[(u'Tom_Holland', 102060),
 (u'cbbaze', 14769),
 (u'ikiya', 12806),
 (u'kr4z33', 9852),
 (u'Chris Lawrence', 9214),
 (u'pdunn', 9067),
 (u'aaront', 8461),
 (u'woodpeck_fixbot', 8378),
 (u'julesreid', 7262),
 (u'bdiscoe', 5095)]

## Number of users who posted only one post

In [64]:
def number_of_users_contributing_once():
    """count the number of users who only contributed once to the map"""
    result = cur.execute("""SELECT COUNT(*)
            FROM 
                (SELECT e.user, COUNT(*) as num
                 FROM (SELECT user FROM nodes UNION ALL SELECT user FROM ways) e
                 GROUP BY e.user
                 HAVING num=1) u""")
    return result.fetchone()[0]
number_of_users_contributing_once()

97

# 3. Additional Data Exploration

## Top 10 amenities

In [62]:
def top_amenities():
    """count the top 10 amentities based on their numbers in the map"""
    amentities = []
    for row in cur.execute("""SELECT value, COUNT(*) as num
                         FROM nodes_tags
                         WHERE key = 'amenity'
                        GROUP BY value
                        ORDER BY num DESC
                        LIMIT 10"""):
        print row[0],":", row[1]
        amentities.append(row)
    #return amentities

top_amenities()

restaurant : 215
fast_food : 108
parking : 73
toilets : 71
cafe : 63
waste_basket : 36
bench : 30
fire_station : 30
parking_entrance : 25
drinking_water : 24


## Types and number of tourism places

In [94]:
def tourism():
    """display the type of tourism places in the map and their numbers"""
    tourism = []
    for row in cur.execute("""SELECT value, COUNT(*) as num
                         FROM nodes_tags
                         WHERE key = 'tourism'
                        GROUP BY value
                        ORDER BY num DESC"""):
        print row[0],":", row[1]
        tourism.append(row)
    #return tourism

tourism()

attraction : 76
viewpoint : 26
hotel : 18
museum : 13
artwork : 10
picnic_site : 8
information : 5
guest_house : 3
hostel : 3
gallery : 2
alpine_hut : 1
camp_site : 1


## If the restaurants allow smoking

In [90]:
def restaurant_smoking_status():
    """display if the restaurants"""
    result = []
    for status in cur.execute("""SELECT nodes_tags.value, COUNT(*) as num
                    FROM nodes_tags 
                    JOIN (SELECT DISTINCT(id) FROM nodes_tags WHERE value='restaurant') i
                    ON nodes_tags.id=i.id
                    WHERE nodes_tags.key='smoking' 
                    GROUP BY nodes_tags.value
                    ORDER BY num DESC"""):
        print status[0], status[1]
        result.append(status)
    
    #return result

restaurant_smoking_status()

no 8
isolated 1


## Sources of information

In [19]:
def source():
    """display the sources of information and the number of times the sources were quoted in the map"""
    source = []
    for row in cur.execute("""SELECT value, COUNT(*) as num
                         FROM nodes_tags
                         WHERE key = 'source'
                        GROUP BY value
                        ORDER BY num DESC"""):
        print row[0],":", row[1]
        source.append(row)
    #return tourism
source()

Bing : 392
USGS Geonames : 116
survey : 42
kr4z33 Survey : 25
Yahoo : 13
OpenStreetBugs : 2
osmsync:dero : 2
ourairports.com : 2
wikipedia : 2
(URL) : 1
Hotel Guest : 1
Landsat : 1
NOAA U.S. Vector Shoreline : 1
Owner : 1
coal : 1
http://jhchawaii.net/ : 1
https://plus.google.com/101962710376509404433/about : 1
tiger_import_dch_v0.6_20070809 : 1


# 4 Conclusion and Reflections

Some observations from the map are as follows.
1) The top1 contributor contributed over 30% of the posts. Top10 contributors contributed 70% of the posts
2) A large part of the data comes from Bing.
3) The information on the amenities, such as resturants are not very complete.
4) For some fields, such as phone number, the input stypes are very inconsistent. 

Suggestions on encouraging data contribution:
1) The open street map can be built into a online community with rewarding systems
2) Functions such as photo uploading can be added.

However, although these methods might increase the data quantity, they might compromise data quality. On one hand, the users might input inaccurate or invalid information just for the rewards or for fun. On the other, the more sources the data come from, the more inconsistent the data might be.

For the issues regarding the validity and consistency of the data, data entry with forced schema or format might help. However, strict restrictions might discourage the users from participating. A less strict schema/format guide might also work.

For the issue regarding data inaccuracy, inviting users to "report inaccuracy" or “revise information” might be help identify and revise the inaccuracy data.

Besides, periodical data cleaning can help reduce the accumulated workload.



# Reference
http://www.sqlitetutorial.net/sqlite-import-csv/
http://www.w3schools.com/sql/sql_syntax.asp
https://stackoverflow.com/questions/19524554/suppress-code-in-nbconvert-ipython
